<img style="float: left;" src="images/logo.png" width=200>

# Does Mobility Make or Break Friendships? - Preprocessing

---
## Introduction

Although our dataset has previously been wrangled and cleaned, we still have to apply a few modifications in able to perform our analysis. In this notebook we will create new datasets to be used in the creation of our prediction models and all of the steps leading up this. In this first step we will introduce the datasets to be used, we will then modify them and finally export new '.csv' files for analyis.

---

## The data

Our extension considers data from a location-based social networks: Foursquare. The datasets are available in the `data` directory pushed to the same GitHub repo as this notebook. This directory contains four '.txt' files and are described below.

### Friendship network data

* `dataset_WWW_friendship_old.txt` Friendship network of Foursquare users in April 2012

* `dataset_WWW_friendship_new.txt.gz` Friendship network of Foursquare users in January 2014

Example:
~~~
[user]      [friendship]
15          595326
19          54
19          1061
...         ...
~~~


### Anonymised check-in data

* `dataset_WWW_Checkins_anonymized.txt` Time and venue ID of check-ins made by Foursquare users.

Example:
~~~
[user]  [venue_ID]                 [checkin_time]                    [offset]
822121  4b4b87b5f964a5204a9f26e3   Tue Apr 03 18:00:07 +0000 2012    -420
208842  4b4606f2f964a520751426e3   Wed Apr 04 11:32:47 +0000 2012    120
113817  3fd66200f964a52000e71ee3   Fri Apr 06 21:30:23 +0000 2012    -300
...     ...                        ...                               ...
~~~


### Venue information data

* `raw_POIs.txt` Venue category and location information of Foursquare venue IDs.

Example:
~~~
[venue_ID]                 [latitude]     [longitude]     [category]           [country_code]
4b4b87b5f964a5204a9f26e3   40.729209	  -73.998753      Mall                 US
4b4606f2f964a520751426e3   36.309536      -119.31340      Asian Restaurant     MX
3fd66200f964a52000e71ee3   -34.28493	  65.3134064      Library              FR
...                        ...            ...             ...                  ...
~~~


---
## Step 1: Loading the data

---
### 1.1 Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import requests

### 1.2  Read the data

In [2]:
# Data folder and dataset location
DATA_FOLDER = 'data/'
FOURSQUARE_FRIENDSHIP_OLD_DATASET = DATA_FOLDER + "dataset_WWW_friendship_old.txt"
FOURSQUARE_FRIENDSHIP_NEW_DATASET = DATA_FOLDER + "dataset_WWW_friendship_new.txt"
FOURSQUARE_CHECKIN_DATASET = DATA_FOLDER + "dataset_WWW_Checkins_anonymized.txt"
FOURSQUARE_VENUE_DATASET = DATA_FOLDER + "raw_POIs.txt"

# Read datasets and create dataframes
friendship_old_df = pd.read_csv(FOURSQUARE_FRIENDSHIP_OLD_DATASET, sep='\t', header=None)
friendship_new_df = pd.read_csv(FOURSQUARE_FRIENDSHIP_NEW_DATASET, sep='\t', header=None)
checkin_df = pd.read_csv(FOURSQUARE_CHECKIN_DATASET, sep='\t', header=None)
venue_df = pd.read_csv(FOURSQUARE_VENUE_DATASET, sep='\t', header=None)

---
### 1.3 Rename and visualise dataframes

In [3]:
# Rename data columns
friendship_old_df.columns = ['user_id', 'friendship_id']
friendship_new_df.columns = ['user_id', 'friendship_id']
checkin_df.columns = ['user_id', 'venue_id', 'checkin_time', 'offset']
venue_df.columns = ['venue_id', 'latitude', 'longitude', 'category', 'country_code']

We visualise the friendship network dataframes

In [4]:
# Visualise dataframe sample
friendship_old_df.sample(3)

,user_id,friendship_id
57908,49184,856881
26016,16617,993796
291441,799332,1839493


In [5]:
# Visualise dataframe sample
friendship_new_df.sample(3)

,user_id,friendship_id
163614,158202,940170
8640,5712,1350060
555086,1248052,1368464


We visualise the anonymised check-in dataframe

In [6]:
# Visualise dataframe sample
checkin_df.sample(3)

,user_id,venue_id,checkin_time,offset
12715912,788345,5066b2bde4b0f646451faa9e,Sat Dec 15 18:25:15 +0000 2012,120
19327244,533669,4fbe5346e4b057d1bdac03df,Tue May 14 14:23:18 +0000 2013,-240
14156762,865631,4bf031f9d4f70f474f97390f,Wed Jan 16 11:04:07 +0000 2013,120


We visualise the venue information dataframe

In [7]:
# Visualise dataframe sample
venue_df.sample(3)

,venue_id,latitude,longitude,category,country_code
8274718,505f7263e4b0be434f8b39c8,38.310618,20.556750,Assisted Living,GR
6621974,4f94c58de4b04c0b8df7ee64,-6.603305,106.776544,Home (private),ID
7868926,5025df38e4b0aa1674cddd07,51.836607,107.583874,Hostel,RU


---

## Step 2: Friendship network preprocessing

---

### 2.1 Remove unwanted users

First we remove users with under 70 checkins, about a third of the mean (199 checkins). 

In [8]:
# Compute average and minimum checkin values
mean_num_checkins = checkin_df['user_id'].value_counts().mean()
min_num_checkins  = checkin_df['user_id'].value_counts().min()

# Print results
print('The average number of checkins per user is %d.' % mean_num_checkins)
print('The minimum number of checkins per user is %d.' % min_num_checkins)

The average number of checkins per user is 199.
The minimum number of checkins per user is 73.


In [9]:
# Global variable for minimum checkin number
MIN_CHECKINS = 70

# For print
size_user0 = len(checkin_df.user_id.unique())
size_checkins0 = checkin_df.shape[0]

# Count number of checkins per user and remove users with under 70 checkins
num_checkins = checkin_df['user_id'].value_counts()
indexes_to_remove = num_checkins[num_checkins < MIN_CHECKINS].index
checkin_df = checkin_df[~checkin_df['user_id'].isin(indexes_to_remove)]

# For print
size_user1 = len(checkin_df.user_id.unique())
size_checkins1 = checkin_df.shape[0]

# Print
print('Users removed: %d' % (size_user1-size_user0))
print('Checkins removed: %d' % (size_checkins1-size_checkins0))

Users removed: 0
Checkins removed: 0


We also want to remove users that are not in both the `dataset_WWW_friendship_old.txt` and `dataset_WWW_friendship_new.txt` datasets.

In [10]:
# Get users not in both dataframes
user_id_old_array = friendship_old_df.user_id.unique()
user_id_new_array = friendship_new_df.user_id.unique()
users_to_remove = np.setxor1d(user_id_old_array, user_id_new_array)

# For print
size_old_user0 = len(user_id_old_array)
size_new_user0 = len(user_id_new_array)

# Remove unwanted users
friendship_old_df = friendship_old_df[~friendship_old_df.user_id.isin(users_to_remove)]
friendship_new_df = friendship_new_df[~friendship_new_df.user_id.isin(users_to_remove)]
checkin_df = checkin_df[~checkin_df.user_id.isin(users_to_remove)]

# For print
size_old_user1 = len(friendship_old_df.user_id.unique())
size_new_user1 = len(friendship_new_df.user_id.unique())

# Print
print('Users removed (old): %d' % (size_old_user0-size_old_user1))
print('Users removed (new): %d' % (size_new_user0-size_new_user1))
print('Users removed (total): %d' % len(users_to_remove))

Users removed (old): 2356
Users removed (new): 15824
Users removed (total): 18180


---
### 2.1 Number of friends before and after

We must first create a new dataframe with friendship information for each user before and after the 22 month period, as well as the number of friendships that user makes or breaks.

In [11]:
# Get number of friends before and after 22 month period
num_before = friendship_old_df.groupby('user_id').count().reset_index()
num_after  = friendship_new_df.groupby('user_id').count().reset_index()

# Merge dataframes on user_id to have friends before and after in same dataframe
friendship_df = num_before.merge(num_after, on='user_id', suffixes=('_before', '_after'), how='inner')

# Rename columns
friendship_df.columns = ['user_id', 'num_friends_before', 'num_friends_after']

# Create column with gained friendships
friendship_df['num_friends_gained'] = friendship_df['num_friends_after'] - friendship_df['num_friends_before']

We visualise the friendship information dataframe.

In [12]:
# Visualise dataframe sample
friendship_df.sample(3)

,user_id,num_friends_before,num_friends_after,num_friends_gained
60457,1183799,3,2,-1
15530,210588,2,6,4
32869,509773,2,2,0


---

## Step 3: Checkin information preprocessing

---

### 3.1 Datetime handling

Our dataset contains the check-in time for each check-in in the following format in two columns: 
* **checkin_time:** %a %b %d %H:%M:%S %z %Y (eg. Tue Apr 03 18:00:07 +0000 2012)
* **offset:** %M (eg. 180)

*(With %a the day, %b the month, %d the date, %H the hour, %M the minute, %S the secind, %Z the zero hour offset and %Y the year)*

We want to remove the offset column and consider it in the check-in time column to get the following format in a single column:
* **checkin_time:** %Y-%m-%d %H:%M:%S (eg. 2012-04-03 21:00:07)

We first notice that one row has a different and unreadable format as shown below.

In [13]:
# Display error row
checkin_df[checkin_df.checkin_time == 'Mon21239673244639234']

,user_id,venue_id,checkin_time,offset
17720249,219528,4dc98660d4c0abe9b6320d4f,Mon21239673244639234,-180


We remove this row.

In [14]:
# Remove error row
checkin_df = checkin_df.drop(index=checkin_df[checkin_df.checkin_time == 'Mon21239673244639234'].index)

Now we modify the check-in time format and add the offset.

In [15]:
# Convert to datetime format
checkin_df['checkin_time'] = pd.to_datetime(checkin_df['checkin_time'], format='%a %b %d %H:%M:%S %z %Y')

# Apply offset
checkin_df['checkin_time'] = checkin_df.apply(lambda x: x.checkin_time + timedelta(minutes=x.offset), axis=1)

# Remove time zone from check-in time and remove offset column
checkin_df['checkin_time'] = checkin_df['checkin_time'].apply(lambda x: x.replace(tzinfo=None))
checkin_df = checkin_df.drop('offset', axis=1)

We visualise the new check-in time dataframe.

In [16]:
# Visualise dataframe sample
checkin_df.sample(3)

,user_id,venue_id,checkin_time
18340770,38935,4bf15b2870779521fc883e7c,2013-04-22 04:51:45
9175291,1463673,4b4bc8c7f964a52091a726e3,2012-09-06 11:55:52
2259503,1692228,4b5fa00af964a52048c529e3,2012-04-29 15:40:46


---

## Step 4: Venue information preprocessing

---

### 4.1 Foursquare API

We have created a foursquare developper account to have acces to the API. Below is our client ID and secret.


In [17]:
# Foursquare ID
CLIENT_ID = '5P0XGYOSTA5D40V4DBIXKOYDPGGYUEC0AQ1RRK22HUXSSKXS'

# Foursquare Secret
CLIENT_SECRET = 'UYHEXP2ZBWO0JE4V3GUQMXYAC0UQZHOSQ1OUSED3Z3EZD1PL'

# Foursquare Vesrsion
VERSION = '20140101'

From our identifications we can request a formatted JSON response from Foursquare giving us information on categories in Foursquare, including their hierarchy and other details.

In [18]:
# URL for json request
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)

# Formatted JSON response
response = requests.get(url).json()

The 'response' object is a dictionary containing all of the hierarchy, naming, opening hours and more details.

---
### 4.1 Foursquare Categories

From the response we will mainly be using the hierarchy of categories to be able to group them. Categories have 5 hierarchy levels. Here is an example:

* Level 0: Food
    * Level 1: African Restaurant
        * Level 2: Ethiopian Restaurant <br />
        ...
        
        <br />
    * Level 1: Asian Restaurant
        * Level 2: Chinese Restaurant
            * Level 3: Anhui Restaurant <br />
        ...
        
        <br />
            
    * Level 1: Latin American Restaurant
        * Level 2: South American Restaurant
            * Level 3: Brazilian Restaurant
                * Level 4: Acai House <br />
        ...
        
  ...
  <br />
* Level 0: Nightlife Spot
    * Level 1: Bar
        * Level 2: Beer Bar <br />
        ...
         

First let us see how many categories we have in the 'venue_df' dataframe.

In [19]:
# Get unique categories
checkin_categories = venue_df.category.unique()

# Print
print('Number of unique categories in dataframe: %d' % len(checkin_categories))

Number of unique categories in dataframe: 519


After analysis, we can see that there are some categories that cannot be found in the JSON response. However, most of these are either slight changes in formulation ('place' instead of 'restaurant') and some have character errors ('�' instead of 'é'). We correct all of these errors, and if the category can really not be found in the response, we add it to a "Other" category.

In [20]:
# Remove exception category names from df
venue_df.category = venue_df.category.replace({'Drugstore / Pharmacy':'Drugstore', 
                                               'Caf��':'Café',
                                               'Caf�':'Café',
                                               'Spa / Massage':'Spa',
                                               'Athletic & Sport':'Athletics & Sports',
                                               'Ramen /  Noodle House':'Ramen',
                                               'Ramen / Noodle House':'Ramen',
                                               'Subway':'Metro Station',
                                               'Hiking Trail':'Trail',
                                               'Car Dealership':'Auto Dealership',
                                               'Stable':'Stables',
                                               'Ferry':'Boat or Ferry',
                                               'Malaysian Restaurant':'Asian Restaurant',
                                               'Martial Arts Dojo':'Martial Arts School',
                                               'Laboratory':'Research Laboratory',
                                               'Frozen Yogurt':'Yogurt',
                                               'Yogurts':'Yogurt',
                                               'Meatball Place':'Italian Restaurant',
                                               'Home Cooking Restaurant':'Other',
                                               'Gas Station / Garage':'Gas Station',
                                               'Kokore? Restaurant': 'Kokoreç Restaurant',
                                               'Kokoretsi Restaurant': 'Kokoreç Restaurant',
                                               'Kofte Restaurant': 'Kofte Place',
                                               })

# Get unique categories
checkin_categories = venue_df.category.unique()

We now want to look through the JSON response to add the category level 0 and level 1 to the venue information dataframe. This will allow us for better grouping.

In [21]:
# Empty arrays to append
level_0_category = []
level_1_category = []

# Flag if category level 1 is found
flag = True

# Level 0 category list
categories_json_0 = response.get('response').get('categories')

# Iterate over check-in categories in dataframe to find hierarchy level 1 and level 0
for c in checkin_categories:
    flag = False

    # Tier 0 
    for i in range(len(categories_json_0)):
        if flag:
            break            
        name0 = categories_json_0[i].get('name')
        name = name0
        plural_name = categories_json_0[i].get('pluralName')
        short_name = categories_json_0[i].get('shortName')
        if (c == name) | (c == plural_name) | (c == short_name):
            level_0_category.append(name0)
            # If category is already level 0, level 1 takes level 0 name
            level_1_category.append(name0)
            flag = True
            break        
        categories_json_1 = categories_json_0[i].get('categories')

        # Tier 1
        for j in range(len(categories_json_1)):
            if flag:
                break    
            name1 = categories_json_1[j].get('name')
            name = name1
            plural_name = categories_json_1[j].get('pluralName')
            short_name = categories_json_1[j].get('shortName')
            if (c == name) | (c == plural_name) | (c == short_name):
                level_0_category.append(name0)
                level_1_category.append(name1)
                flag = True
                break           
            categories_json_2 = categories_json_1[j].get('categories')

            # Tier 2
            for k in range(len(categories_json_2)):
                if flag:
                    break
                name = categories_json_2[k].get('name')
                plural_name = categories_json_2[k].get('pluralName')
                short_name = categories_json_2[k].get('shortName')
                if (c == name) | (c == plural_name) | (c == short_name):
                    level_0_category.append(name0)
                    level_1_category.append(name1)
                    flag = True
                    break                
                categories_json_3 = categories_json_2[k].get('categories')

                # Tier 3
                for l in range(len(categories_json_3)):
                    if flag:
                        break
                    name = categories_json_3[l].get('name')
                    plural_name = categories_json_3[l].get('pluralName')
                    short_name = categories_json_3[l].get('shortName')
                    if (c == name) | (c == plural_name) | (c == short_name):
                        level_0_category.append(name0)
                        level_1_category.append(name1)
                        flag = True
                        break
                    categories_json_4 = categories_json_3[l].get('categories')

                    # Tier 4
                    for m in range(len(categories_json_4)):
                        if flag:
                            break 
                        name = categories_json_4[m].get('name')
                        plural_name = categories_json_4[m].get('pluralName')
                        short_name = categories_json_4[m].get('shortName')
                        if (c == name) | (c == plural_name) | (c == short_name):
                            level_0_category.append(name0)
                            level_1_category.append(name1)
                            flag = True
                            break
                            
    # Return category name if not found in Foursquare API response
    if not flag:
        level_0_category.append(c)
        level_1_category.append(c)

We create a new datframe showing each category, its level 0 and its level 1 category.

In [22]:
# Create dataframe from arrays
category_df = pd.DataFrame({'category':checkin_categories, 
                            'level_0_category':level_0_category, 
                            'level_1_category':level_1_category})

We visualise the category dataferame.

In [23]:
# Visualise sample of dataframe
category_df.sample(3)

,category,level_0_category,level_1_category
188,Airport,Travel & Transport,Airport
347,Water Park,Arts & Entertainment,Water Park
341,Bed & Breakfast,Travel & Transport,Hotel


From the Foursquare API we know that we have 10 level 0 categories. We have also added the "Other" category, so we should have a total of 11 level 0 categories.

In [24]:
# Print number of level 0 categories
print('Number of unique level 0 categories: %d' % len(category_df.level_0_category.unique()))

# Display level 0 categories
category_df.level_0_category.unique()

Number of unique level 0 categories: 11


array(['Professional & Other Places', 'Arts & Entertainment',
       'College & University', 'Food', 'Nightlife Spot', 'Shop & Service',
       'Travel & Transport', 'Outdoors & Recreation', 'Residence',
       'Event', 'Other'], dtype=object)

Finally, we merge the venue information dataframe with the hierarchy category level dataframe.

In [25]:
# Merge dataframes
venue_df = pd.merge(venue_df, category_df, on='category')

We visualise venue information dataframe.

In [26]:
# Visualise dataframe sample 
venue_df.sample(3)

,venue_id,latitude,longitude,category,country_code,level_0_category,level_1_category
7092355,50fbcac8e4b09f8ff685473b,-6.881669,107.615613,Home (private),ID,Residence,Home (private)
6835229,50267cfae4b04e431bbc170a,10.294478,123.880195,Home (private),PH,Residence,Home (private)
9670452,4ce0ef7441ed224b54a2e73c,39.477303,-0.442483,Automotive Shop,ES,Shop & Service,Automotive Shop


---

## Step 5: Exporting the data

In this finals step, we export the preprocessed dataframes as `.csv` files for analysis.

In [27]:
# Friendship information dataframe
friendship_df.to_csv('friendship_df.csv.zip',
                     index=False,
                     compression=dict(method='zip', archive_name='friendship_df.csv'))

# Friendship information dataframe
checkin_df.to_csv('checkin_df.csv.zip',
                  index=False,
                  compression=dict(method='zip', archive_name='checkin_df.csv'))

# Venue information dataframe
venue_df.to_csv('venue_df.csv.zip',
          index=False,
          compression=dict(method='zip', archive_name='venue_df.csv'))

---